<a href="https://colab.research.google.com/github/malinphy/Google_bert/blob/main/imdb_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet tensorflow-text
import numpy as np 

import pandas as pd
import tensorflow_hub as hub
import os 
import tensorflow as tf 
import tensorflow_text as text
from tensorflow import keras 
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import seaborn as sns 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/SrinidhiRaghavan/AI-Sentiment-Analysis-on-IMDB-Dataset/master/imdb_tr.csv',encoding = "ISO-8859-1")

In [ ]:
print(len(data))
data.head(2)

25000


,row_Number,text,polarity
0,2148,"first think another Disney movie, might good, ...",1
1,23577,"Put aside Dr. House repeat missed, Desperate H...",0


In [ ]:
data['polarity'].value_counts()

1    12500
0    12500
Name: polarity, dtype: int64

In [ ]:
first_train, first_test = train_test_split(data, test_size=0.2, train_size= 0.8, random_state = 42)
second_train, second_test = train_test_split(first_train, test_size=0.1, train_size= 0.9, random_state = 42)

In [ ]:
len(second_test)

2000

In [ ]:
bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8'  

tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'
tfhub_handle_preprocess =  hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
bert_model = hub.KerasLayer(tfhub_handle_encoder)

In [ ]:
input_layer = tf.keras.layers.Input(shape = (), dtype = tf.string)
preprocess_layer = tfhub_handle_preprocess(input_layer)
bert_layer = bert_model(preprocess_layer)
drop_layer= tf.keras.layers.Dropout(0.2)(bert_layer['pooled_output'])
output_layer = tf.keras.layers.Dense(1, activation = None)(drop_layer)
bert_model = tf.keras.Model(inputs= input_layer, outputs = output_layer)

In [ ]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = tf.metrics.BinaryAccuracy()
optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
    name='Adam', 
)

bert_model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)



In [ ]:
bert_model.fit(first_test['text'],first_test['polarity'],
               epochs = 1)

157/157 [==============================] - 40s 205ms/step - loss: 0.6614 - binary_accuracy: 0.5770


In [ ]:
preds = bert_model.predict(pred_res)

In [ ]:
pred_res = []
for i in preds:
  if i >0.5:
    pred_res.append(1)
  if i<0.5:
    pred_res.append(0)

In [ ]:
tf.math.confusion_matrix(
    pred_res, pred_res, num_classes=None, weights=None, dtype=tf.dtypes.int32,
    name=None
)


<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[1399,    0],
       [   0,  601]], dtype=int32)>